In [5]:
%pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.1 MB/s eta 0:00:00


In [6]:
from transformers import MarianMTModel, MarianTokenizer, MarianConfig

tokenizer = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-de-en")
model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-de-en")
config = MarianConfig.from_pretrained("Helsinki-NLP/opus-mt-de-en")

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/797k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/768k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/298M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [7]:
import torch
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [8]:
import torch.optim as optim
from torch.nn import CrossEntropyLoss
from torch.optim.lr_scheduler import StepLR

model.to(device)

optimizer = optim.Adam(model.parameters(), lr=1e-5)

# criterion = CrossEntropyLoss()

scheduler = StepLR(optimizer, step_size=5, gamma=0.5)


In [9]:
import nltk
from nltk.translate.bleu_score import corpus_bleu, sentence_bleu

In [6]:
%pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.8 MB/s eta 0:00:00


In [7]:
from datasets import load_dataset
import pandas as pd
multi30k = load_dataset("bentrevett/multi30k")
# train_data, valid_data, test_data = pd.DataFrame(multi30k['train']), pd.DataFrame(multi30k['validation']), pd.DataFrame(multi30k['test'])
train_data, valid_data, test_data = multi30k['train'], multi30k['validation'], multi30k['test']


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
train_tokenized = tokenizer(list(train_data["de"]), text_target=list(train_data["en"]), return_tensors="pt", padding=True, truncation=True, add_special_tokens=True).to(device)

In [ ]:
test_tokenized = tokenizer(list(test_data["de"]), text_target=list(test_data["en"]), return_tensors="pt", padding=True, truncation=True, add_special_tokens=True).to(device)
valid_tokenized = tokenizer(list(valid_data["de"]), text_target=list(valid_data["en"]), return_tensors="pt", padding=True, truncation=True, add_special_tokens=True).to(device)


In [ ]:
from torch.utils.data import TensorDataset, DataLoader

torch.manual_seed(1)
data_set = TensorDataset(train_tokenized.input_ids, train_tokenized.attention_mask, train_tokenized.labels)
data_loader = DataLoader(data_set, batch_size=128)


In [ ]:
references = train_data["en"]

In [ ]:
model.eval()
output_sentences = []
i = 0

for batch in data_loader:
    i+=1
    print(i, '/', len(data_loader))
    output_ids = model.generate(
        input_ids = batch[0].to(device),
        attention_mask = batch[1].to(device),
        labels = batch[2].to(device))
    output_sentences.extend(tokenizer.batch_decode(output_ids, skip_special_tokens=True))

# 10min34s, 14.4G/15G


In [ ]:
import spacy
spacy_en = spacy.load('en_core_web_sm')
def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [ ]:
hypotheses = [tokenize_en(s) for s in output_sentences]
hypotheses[0]

['Two',
 'young',
 'white',
 'men',
 'are',
 'outdoors',
 'near',
 'many',
 'bushes',
 '.']

In [ ]:
references = [tokenize_en(s) for s in references]
references[0]

['Two',
 'young',
 ',',
 'White',
 'males',
 'are',
 'outside',
 'near',
 'many',
 'bushes',
 '.']

In [ ]:
sentence_bleu([references[0]], hypotheses[0])

0.29377167835760565

In [ ]:
bleu_scores = []
for i, hypothesis in enumerate(hypotheses):
    bleu_score = sentence_bleu(references=[references[i]], hypothesis=hypothesis)
    bleu_scores.append(round(bleu_score, 4))

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

In [ ]:
torch.save(bleu_scores, 'bleu_scores_r4')

In [10]:
bleu_scores = torch.load('bleu_scores_r4')

In [11]:
bleu_scores = torch.Tensor(bleu_scores) * 100

In [ ]:
def map_bleu_scores_to_tokens(bleu_scores):

    categories_ranges = list(range(5, 110, 10)) # [5, 15, 25, 35, 45, 55, 65, 75, 85, 95, 105]

    BLEU_tokens = []

    for bleu_score in bleu_scores:
        for i, upper_bound in enumerate(categories_ranges):
            if bleu_score <= upper_bound:
                BLEU_tokens.append(f'BLEU_{upper_bound-5}')
                break

    return BLEU_tokens

def map_bleu_scores_to_tokens_5categories(bleu_scores):

    categories_ranges = list(range(20, 110, 20)) # [20, 40, 60, 80, 100]

    BLEU_tokens = []

    for bleu_score in bleu_scores:
        for i, upper_bound in enumerate(categories_ranges):
            if bleu_score <= upper_bound:
                BLEU_tokens.append(f'bleu_{upper_bound-20}_{upper_bound}')
                break

    return BLEU_tokens

# bleu_scores = [3, 7, 18, 30, 42, 52, 62, 72, 82, 92, 100]
# BLEU_tokens = map_bleu_scores_to_tokens(bleu_scores)
BLEU_tokens = map_bleu_scores_to_tokens_5categories(bleu_scores)


for bleu_score, category in zip(bleu_scores, BLEU_tokens):
    print(f'bleu_{bleu_score}: {category}')


In [4]:
list(range(20, 110, 20)) # [5, 15, 25, 35, 45, 55, 65, 75, 85, 95, 105]


[20, 40, 60, 80, 100]

In [ ]:
idx = bleu_scores.index(0)
hypotheses[idx], references[idx]

(['Several',
  'men',
  'with',
  'protective',
  'helmets',
  'operate',
  'a',
  'drive',
  'wheel',
  'system',
  '.'],
 ['Several',
  'men',
  'in',
  'hard',
  'hats',
  'are',
  'operating',
  'a',
  'giant',
  'pulley',
  'system',
  '.'])

In [ ]:
torch.save(output_sentences, 'Targets_T_multi30k.pth')

In [18]:
torch.save(BLEU_tokens, 'bleu_tokens_c5')

In [14]:
BLEU_tokens[:10]

['bleu_(20, 40]',
 'bleu_(0, 20]',
 'bleu_(40, 60]',
 'bleu_(60, 80]',
 'bleu_(0, 20]',
 'bleu_(60, 80]',
 'bleu_(40, 60]',
 'bleu_(0, 20]',
 'bleu_(40, 60]',
 'bleu_(60, 80]']